## Logistic Regression using 1 layer NN

In [873]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Data Preparation

In [874]:
## Load the data first
train_org  = h5py.File('./datasets/train_catvnoncat.h5')
test_org = h5py.File('./datasets/test_catvnoncat.h5')

In [875]:
## set the keys available in train and test data
print("Train data keys :",[key for key in train_org.keys()])
print("Test data keys :",[key for key in test_org.keys()])

Train data keys : ['list_classes', 'train_set_x', 'train_set_y']
Test data keys : ['list_classes', 'test_set_x', 'test_set_y']


In [876]:
train_org_x = np.array(train_org['train_set_x'])
train_org_y = np.array(train_org['train_set_y'])

test_org_x = np.array(test_org['test_set_x'])
test_org_y = np.array(test_org['test_set_y'])

list_classes = np.array(train_org['list_classes'])

In [877]:
## Check the shape of train and test data
print("Train X shape :",train_org_x.shape)
print("Train Y shape :",train_org_y.shape)
print("Test X shape :",test_org_x.shape)
print("Test Y shape :",test_org_y.shape)
print("Classes :", list_classes)

Train X shape : (209, 64, 64, 3)
Train Y shape : (209,)
Test X shape : (50, 64, 64, 3)
Test Y shape : (50,)
Classes : [b'non-cat' b'cat']


In [878]:
## Create feature vector
train_x  = train_org_x.reshape(-1,train_org_x.shape[0])
train_y  = train_org_y.reshape(1,train_org_y.shape[0])

test_x  = test_org_x.reshape(-1,test_org_x.shape[0])
test_y  = test_org_y.reshape(1,test_org_y.shape[0])

In [879]:
## Again check the shape 
print("Train X shape :",train_x.shape)
print("Train Y shape :",train_y.shape)

print("Test X shape :",test_x.shape)
print("Test Y shape :",test_y.shape)

Train X shape : (12288, 209)
Train Y shape : (1, 209)
Test X shape : (12288, 50)
Test Y shape : (1, 50)


In [880]:
## normalize the input vectors 
train_x = train_x/255
test_x = test_x/255

## 1 layer NN

In [881]:
def sigmoid(z):
    return 1. /(1+np.exp(-z))

In [882]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))

sigmoid([0, 2]) = [0.5        0.88079708]


In [883]:
def initialize_params(x):
    
    w = np.zeros(shape=(x.shape[0], 1), dtype=np.float32)
    #w = np.random.randn(x.shape[0]) * .005
    b = 0
    
    return w,b

In [884]:
def forward_prop(x,y,w,b):
    
    m = x.shape[1]
    z = np.dot(w.T,x) + b
    a = sigmoid(z)
    
    cost = (- 1./m) * np.sum((np.sum((y * np.log(a) + (1-y) * np.log(1-a)))))
    
    forward_params = {"a":a,"z":z,"cost":cost}
    
    return forward_params

In [885]:
def backward_prop(x,y,params):
    
    a = params['a']
    m = x.shape[1]
    
    dz = a-y
    dw = (1./m) * np.dot(x,dz.T)
    db = (1./m) * np.sum(dz)
    
    grads = {"dz":dz,"dw":dw,"db":db}
    
    return grads

In [886]:
def gradient_optimizer(w,b,learning_rate,grads):
    
    dw = grads["dw"]
    db = grads["db"]

    w = w - learning_rate * dw
    b = b - learning_rate  * db
    
    return w,b

In [887]:
def predict(x,w,b,threshold=.5):
    
    # have to reshape w 
    #w = w.reshape(x.shape[0], 1)
    
    z = np.dot(w.T,x) + b
    a = sigmoid(z)
    
    predicted = np.where(a>=threshold,1,0)
    
    return predicted

In [888]:
def model(x,y,test_x, test_y,learning_rate, n_iteration = 500, printFlag = False):
    
    # intialie params
    w,b = initialize_params(x)

    for i in range(n_iteration):
        # forward prop
        f_params = forward_prop(x,y,w,b)
        # print cost
        if i%100 == 0 and printFlag:
            print("Cost after",i,"iteration is:",f_params['cost'])
        # backward_prop
        grads = backward_prop(x,y,f_params)
        # gradient_optimizer
        w,b = gradient_optimizer(w,b,learning_rate=learning_rate,grads=grads)
    
    train_predicted = predict(x,w,b,threshold=0.5)
    test_predicted = predict(test_x,w,b,threshold=0.5)
    
    print("Train accuracy:",(1 - np.mean(np.abs(train_predicted - y))) * 100)
    print("Test accuracy:",(1- np.mean(np.abs(test_predicted - test_y))) * 100)

In [889]:
model(x=train_x,
      y=train_y,
      test_x=test_x,
      test_y=test_y,
      learning_rate=.004,
      n_iteration=2000,
      printFlag=True)

Cost after 0 iteration is: 0.6931471805599453
Cost after 100 iteration is: 0.5714612828790993
Cost after 200 iteration is: 0.5375611803973772
Cost after 300 iteration is: 0.5156860641264179
Cost after 400 iteration is: 0.4977808983845982
Cost after 500 iteration is: 0.48233253270691473
Cost after 600 iteration is: 0.46862918905943585
Cost after 700 iteration is: 0.45626516897838626
Cost after 800 iteration is: 0.44497709222740595
Cost after 900 iteration is: 0.4345791594204344
Cost after 1000 iteration is: 0.42493307550704384
Cost after 1100 iteration is: 0.4159320717104525
Cost after 1200 iteration is: 0.4074915062662864
Cost after 1300 iteration is: 0.39954290779708224
Cost after 1400 iteration is: 0.3920299878800363
Cost after 1500 iteration is: 0.3849058583780306
Cost after 1600 iteration is: 0.37813102483090266
Cost after 1700 iteration is: 0.3716719005788175
Cost after 1800 iteration is: 0.36549968216631457
Cost after 1900 iteration is: 0.35958948251703227
Train accuracy: 87.5598

### Keras Implementation

In [900]:
train_x.shape

(12288, 209)

In [923]:
from keras.models import  Sequential
from keras.layers import Dense, Activation
import keras

In [924]:
model = Sequential()

In [926]:
model.add(Dense(units=1, activation= keras.activations.sigmoid, input_dim = train_x.shape ))

TypeError: Error converting shape to a TensorShape: int() argument must be a string, a bytes-like object or a number, not 'tuple'.

In [ ]:
model.compile(optimizer='sgd', loss= keras.losses.binary_crossentropy, metrics=['accuracy'])

In [922]:
model.fit(x = train_x, y= train_y)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.